# BMDS for ToxRefDB - Part 3 - Uploading Results to ToxRefDB
The objectives of this document is to upload the tables and output from the BMDS analysis into the ToxRefDB_2_0 database.

In [1]:
library(RMySQL)
library(data.table)
library(magrittr)
library(readxl)


setwd("")

con <- dbConnect(drv = RMySQL::MySQL(), group = "toxrefdb_2_0")


Loading required package: DBI


In [3]:
cancer_tr <- read.csv("cancer_tr.csv") %>%
data.table() %>%
.[ , data_type:="cancer"]

dichotomous <- read.csv("dichotomous_tr.csv") %>%
data.table() %>%
.[ , data_type:="dichotomous"]

continuous_notBW <- read.csv("continuous_NotBW.csv") %>%
data.table() %>%
.[ , data_type:="continuous_notBW"]

continuous_BW <- read.csv("continuous_BW.csv") %>%
data.table() %>%
.[ , data_type:="continuous_BW"]

bmd_continuous_input <- rbind(continuous_notBW, continuous_BW)
bmd_dichotomous_input <- rbind(cancer_tr, dichotomous)


In [7]:
dbWriteTable(con, "bmd_continuous_input", bmd_continuous_input, overwrite=TRUE)
dbWriteTable(con, "bmd_dichotomous_input", bmd_dichotomous_input, overwrite=TRUE)

[1] TRUE

[1] TRUE

In [ ]:
# Importing Files

ToxRefDB_bmd_continuous_1sd <- read_excel("ToxRefDB_bmd_continuous_1sd.xlsx") %>% 
data.table() %>%
.[ , bmr:=1] %>%
.[ , bmr_type:="sd"] %>%
.[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(dataset_id, "_", fixed=TRUE)]

ToxRefDB_bmd_continuousBW_10rd <- read_excel("ToxRefDB_bmd_continuousBW_10rd.xlsx") %>% 
data.table() %>%
.[ , bmr:=10] %>%
.[ , bmr_type:="rd"] %>%
.[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(dataset_id, "_", fixed=TRUE)]

ToxRefDB_bmd_dichotomous_05bmr <- read_excel("ToxRefDB_bmd_dichotomous_05bmr.xlsx") %>% 
data.table() %>%
.[ , bmr:=5] %>%
.[ , bmr_type:="bmr"] %>%
.[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(dataset_id, "_", fixed=TRUE)]

ToxRefDB_bmd_dichotomous_10bmr <- read_excel("ToxRefDB_bmd_dichotomous_10bmr.xlsx") %>% 
data.table() %>%
.[ , bmr:=10] %>%
.[ , bmr_type:="bmr"] %>%
.[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(dataset_id, "_", fixed=TRUE)]

ToxRefDB_bmd_dichotomous_cancer5bmr <- read_excel("ToxRefDB_bmd_dichotomous_cancer5bmr.xlsx") %>% 
data.table() %>%
.[ , bmr:=5] %>%
.[ , bmr_type:="bmr"] %>%
.[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(dataset_id, "_", fixed=TRUE)]

ToxRefDB_bmd_dichotomous_cancer10bmr <- read_excel("ToxRefDB_bmd_dichotomous_cancer10bmr.xlsx") %>% 
data.table() %>%
.[ , bmr:=10] %>%
.[ , bmr_type:="bmr"] %>%
.[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(dataset_id, "_", fixed=TRUE)]

bmd_table <-rbind(ToxRefDB_bmd_continuous_1sd,
                  ToxRefDB_bmd_continuousBW_10rd,
                  ToxRefDB_bmd_dichotomous_05bmr,
                  ToxRefDB_bmd_dichotomous_10bmr,
                  ToxRefDB_bmd_dichotomous_cancer5bmr,
                  ToxRefDB_bmd_dichotomous_cancer10bmr) %>%
.[ ,recommended:=as.numeric(recommended)]

bmd_table[ , dfile:=NULL]
bmd_table[ , outfile:=NULL]

In [4]:
head(bmd_table)

dataset_id,doses_dropped,model_name,model_version,has_output,BMD,BMDL,BMDU,CSF,AIC,...,logic_cautions,logic_warnings,logic_failures,recommended,recommended_variable,bmr,bmr_type,study_id,endpoint_id,tg_effect_id
1006_187_1263,0,Exponential-M2,1.10,TRUE,272.8610,213.5970,-999,-999,125.149,...,NA,NA,NA,0,NA,1,sd,1006,187,1263
1006_187_1263,0,Exponential-M3,1.10,TRUE,274.6260,213.6550,-999,-999,127.1413,...,NA,NA,NA,0,NA,1,sd,1006,187,1263
1006_187_1263,0,Exponential-M4,1.10,TRUE,268.0060,116.7010,-999,-999,127.2147,...,NA,NA,NA,0,NA,1,sd,1006,187,1263
1006_187_1263,0,Exponential-M5,1.10,TRUE,86.1424,71.1897,-999,-999,129.0549,...,NA,NA,NA,1,BMDL,1,sd,1006,187,1263
1006_187_1263,0,Hill,2.17,TRUE,98.4164,73.0344,-999,-999,129.054913,...,NA,NA,NA,0,NA,1,sd,1006,187,1263
1006_187_1263,0,Linear,2.20,TRUE,268.0310,182.5720,-999,-999,125.213939,...,NA,NA,NA,0,NA,1,sd,1006,187,1263


In [ ]:
dbWriteTable(con, "bmd_models", bmd_table, overwrite=TRUE)